In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
# get root directory
import os
root_dir = os.getcwd()
# get the path to the data directory
data_dir = os.path.join(root_dir, 'data')


In [15]:
from langchain.document_loaders import Pdf

'/Users/phamanhdung/Documents/AI/extract_data_from_pdf/extract_data_from_pdf/data'